In [1]:
import os, sys
import pandas as pd
import numpy as np
import xarray as xr
from itertools import product
from datetime import datetime, timedelta
import argparse
import logging
from scipy.special import gamma, factorial #gamma function
sys.path.append('/g/data/er4/zk6340/code/lmoments3')
#import lmoments3 as lm #calculate l-moments
%matplotlib inline

In [ ]:
## dir folder path
dir_in = '/g/data/er4/zk6340/Hydro_projection/data_flood_scenario_pr'
dir_out = '/g/data/er4/zk6340/code/Script_Hydro-projections'

# parameter
parameter = 'pr'

# Which cluster
clusters = {'CS':1,'EC':2,'MB':4,'MN':5,'R':6,'SS':7,'SSWF':8,'WT':9}
which_cluster = clusters['WT']

# return period
yT = 20

# projection period
yr_st = '20800101'
yr_end = '20991231'

# Participating GCMs, Bias correction approaches, Emission scenarios
gcms = ['CNRM-CERFACS-CNRM-CM5','CSIRO-BOM-ACCESS1-0','MIROC-MIROC5','NOAA-GFDL-GFDL-ESM2M']
bias_corr = ['CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP','r240x120-ISIMIP2b-AWAP', 'r240x120-MRNBC-AWAP', 'r240x120-QME-AWAP']
emission = ['rcp45','rcp85']

In [ ]:
#Function to get to the file
def get_file(parameter,which_gcm,which_emission,which_bias_corr, which_metric, yr_end, yr_st):
    base_filename = '%s_AUS-5_%s_%s_r1i1p1_%s_%s_%s-%s.nc'%(parameter,which_gcm,which_emission,which_bias_corr,which_metric,yr_end,yr_st)
    return(base_filename)

In [ ]:
#Function to calculate return period: EV1/Gumble distribution
def return_period(data, T):
    xbar=np.mean(data)
    stdev=np.std(data)
    ff=np.sqrt(6)/3.1416*(-0.5772-np.log(-np.log(1-1/T))) 
    qt=xbar+stdev*ff
    return(qt)

In [ ]:
#Function to calculate return period: GEV distribution
def gev_return_period(data, T):
    lamda1= lm.lmom_ratios(data, nmom=4)[0]
    lamda2= lm.lmom_ratios(data, nmom=4)[1]
    lamda3= lm.lmom_ratios(data, nmom=4)[2]
    lamda4= lm.lmom_ratios(data, nmom=4)[3]
    c=(2*lamda2/(lamda3+3*lamda2))-(np.log(2)/np.log(3))
    k=7.859*c+2.9554*c**2
    alpha=k*lamda2/(gamma(1+k)*(1-2**(-k)))
    zeta=lamda1-alpha/k*(1-(gamma(1+k)))
    qt=zeta+(alpha/k)*(1-(-np.log(1-1/T))**k)
    return(qt)

In [ ]:
#Function to get annual maximum timeseries
def ts_data(dataset, variable, x, cluster):
    tm = dataset.time.dt.strftime('%Y-%m-%d')[x].item(0)
    ds = dataset[variable].sel(time = tm).where(mask == cluster).NRM_cluster.mean().item(0)   
    return(ds)

In [ ]:
# Read cluster mask
mask = xr.open_dataset(os.path.join(dir_in,'NRM_clusters.nc'))

In [ ]:
values = [(i, k, x) for i, k, x in product(gcms,emission, bias_corr)]

In [2]:
x = [ 0.29848486,  0.09815075,  0.10894123,  0.17728981,  0.07993447,
        0.05687778,  0.03262357,  0.08426319,  0.11417754,  2.83034754,
        0.31669611,  0.97379208,  0.3431657 ,  0.43425098,  0.3285206 ,
        0.15549701,  0.3161217 ,  0.22977819,  0.2376546 ,  2.58493233,
        0.86850059,  0.32189187, 12.81156254,  0.61593449,  0.54709554,
        0.27335545,  0.28320321,  0.30237091,  0.3749817 ,  0.27856681]

In [3]:
# Calculates the L/LH moments: 
# Note: L-moments when h = 0 
def lh(x, h):
    lh1 = 0
    lh2 = 0
    lh3 = 0
    lh4 = 0
    ts = np.sort(x)
    n = len(ts)
    for i in range (0, len(ts)): 
        cl0 = 1
        if h>0:
            for j in range (0, h): 
                cl0 = cl0*(i-j)/(j+1)
        cl1 = cl0*(i+1-h-1)/(h+1)
        cl2 = cl1*(i+1-h-2)/(h+2)
        cl3 = cl2*(i+1-h-3)/(h+3)
        cr1 = n-(i+1)
        cr2 = cr1*(n-(i+1)-1)/2
        cr3 = cr2*(n-(i+1)-2)/3
        lh1 = lh1+cl0* ts[i]
        lh2 = lh2+(cl1-cl0*cr1)* ts[i]
        lh3 = lh3+(cl2-2*cl1*cr1+cl0*cr2)* ts[i]
        lh4 = lh4+(cl3-3*cl2*cr1+3*cl1*cr2-cl0*cr3)* ts[i] 
    c0 = 1
    if h>0:
        for j in range (0, h): 
            c0 = c0*(n+1-(j+1))/(j+1)
    c1 = c0*(n+1-h-1)/(h+1)
    c2 = c1*(n+1-h-2)/(h+2)
    c3 = c2*(n+1-h-3)/(h+3)
    c4 = c3*(n+1-h-4)/(h+4)
    lh1 = lh1/c1
    lh2 = lh2/c2/2
    lh3 = lh3/c3/3
    lh4 = lh4/c4/4
    return(lh1,lh2,lh3,lh4)

In [16]:
lh(x, 25)

(11.475757663604451, 4.53051933928202, 2.9479301945900382, 2.587149373916667)

In [ ]:
data_awap = []
for i in range (0, len(x)): 
    cl0 = 1
    if(h>0):
        for j in range (0, h): 
            cl0 = cl0*(i-j)/(j+1)
    else:
        cl1 = cl0*(i+1-h-1)/(h+1)
        cl2 = cl1*(i+1-h-2)/(h+2)
        cl3 = cl2*(i+1-h-3)/(h+3)
        cr1 = n-(i+1)
        cr2 = cr1*(n-(i+1)-1)/2
        cr3 = cr2*(n-(i+1)-2)/3
        lh1 = lh1+cl0* x[i]
        lh2 = lh2+(cl1-cl0*cr1)* x[i]
        lh3 = lh3+(cl2-2*cl1*cr1+cl0*cr2)* x[i]
        lh4 = lh4+(cl3-3*cl2*cr1+3*cl1*cr2-cl0*cr3)* x[i]

In [ ]:
subroutine lh(x, n, h, lh1, lh2, lh3, lh4)
!  Calculates the LH moments
   implicit none
   integer :: n, i, j, h
   real :: x(200), cl0, cl1, cl2, cl3, cr1, cr2, cr3,c0, c1, c2, c3, c4 
   real(8) :: lh1, lh2, lh3, lh4       
   lh1 = 0.; lh2 = 0.; lh3 = 0.; lh4 = 0.
   
    
    do i =1, n
      cl0 = 1.
         if (h > 0) then
         do j = 1, h
            cl0 = cl0*(i-j)/j
         end do
      end if
      cl1 = cl0*(i-h-1)/(h+1)
      cl2 = cl1*(i-h-2)/(h+2)
      cl3 = cl2*(i-h-3)/(h+3)
      cr1 = n-i
      cr2 = cr1*(n-i-1)/2
      cr3 = cr2*(n-i-2)/3
      lh1 = lh1+cl0* x(i)
      lh2 = lh2+(cl1-cl0*cr1)* x(i)
      lh3 = lh3+(cl2-2*cl1*cr1+cl0*cr2)* x(i)
      lh4 = lh4+(cl3-3*cl2*cr1+3*cl1*cr2-cl0*cr3)* x(i)
   end do
   c0 = 1
   if (h > 0) then
      do j = 1, h
         c0 = c0*(n+1-j)/j
      end do
   end if
   c1 = c0*(n+1-h-1)/(h+1)
   c2 = c1*(n+1-h-2)/(h+2)
   c3 = c2*(n+1-h-3)/(h+3)
   c4 = c3*(n+1-h-4)/(h+4)
   lh1 = lh1/c1
   lh2 = lh2/c2/2
   lh3 = lh3/c3/3
   lh4 = lh4/c4/4
end subroutine lh

In [ ]:
# AWAP Max
ds_awap_annual_max = xr.open_dataset(os.path.join(dir_in,'awap_annual_max.nc'))
ds_awap_annual_max_renamed = ds_awap_annual_max.rename({'longitude':'lon','latitude':'lat'})

In [ ]:
# Extract annual maximum timeseries data for a cluster
data_awap = []
for i in range (0, len(ds_awap_annual_max_renamed.time)): 
    annual_max = ts_data(ds_awap_annual_max_renamed, 'rain_day', i, which_cluster)
    data_awap.append(annual_max)

In [ ]:
ts_awap = pd.DataFrame(data_awap, columns = ['awap'])

In [ ]:
return_period(ts_awap, yT)[0]

In [ ]:
gev_return_period(ts_awap['awap'].values, yT)

In [ ]:
# AWAP Mean
ds_awap_mean = xr.open_dataset(os.path.join(dir_in,'awap_mean.nc'))
ds_awap_mean_renamed = ds_awap_mean.rename({'longitude':'lon','latitude':'lat'})
if parameter=='pr':
    ds_awap_cluster_mean = ds_awap_mean_renamed['rain_day'].where(mask == which_cluster)
else:
    ds_awap_cluster_mean = ds_awap_mean_renamed['qtot'].where(mask == which_cluster)
#ds_awap_cluster_mean_avg = ds_awap_cluster_mean.NRM_cluster.mean().item(0)

In [ ]:
ds_awap_annual_max_renamed

In [ ]:
grids = mask.where(mask == which_cluster).NRM_cluster.to_dataframe().reset_index()
grids.dropna(axis=0)
#.set_index(['lon', 'lat'], drop=False)

In [ ]:
grids

In [ ]:
mask.where(mask == which_cluster).NRM_cluster.plot()

In [ ]:
ds_awap_annual_max_masked = ds_awap_annual_max_renamed['rain_day'].where(mask == which_cluster)

In [ ]:
ds_awap_annual_max_masked

In [ ]:
df_ds_awap_annual_max_masked = ds_awap_annual_max_masked.NRM_cluster.to_dataframe().reset_index(drop = True).set_index(['lon', 'lat'], drop=False)
df_ds_awap_annual_max_masked.dropna(axis=0)

In [ ]:
gev_return_period(ts_awap['awap'].values, yT)

In [ ]:
# AWAP Mean
ds_awap_mean = xr.open_dataset(os.path.join(dir_in,'awap_mean.nc'))
ds_awap_mean_renamed = ds_awap_mean.rename({'longitude':'lon','latitude':'lat'})
if parameter=='pr':
    ds_awap_cluster_mean = ds_awap_mean_renamed['rain_day'].where(mask == which_cluster)
else:
    ds_awap_cluster_mean = ds_awap_mean_renamed['qtot'].where(mask == which_cluster)
ds_awap_cluster_mean_avg = ds_awap_cluster_mean.NRM_cluster.mean().item(0)

# AWAP Max
ds_awap_max = xr.open_dataset(os.path.join(dir_in,'awap_max.nc'))
ds_awap_max_renamed = ds_awap_max.rename({'longitude':'lon','latitude':'lat'})
if parameter=='pr':
    ds_awap_cluster_max = ds_awap_max_renamed['rain_day'].where(mask == which_cluster)
else:
    ds_awap_cluster_max = ds_awap_max_renamed['qtot'].where(mask == which_cluster)
ds_awap_cluster_max_avg = ds_awap_cluster_max.NRM_cluster.mean().item(0)

In [ ]:
cluster_flood_indicator=[]
for val in enumerate(values):   
#for val in enumerate(values[0]):  
    #Change in annual mean precipitation/runoff
    filename_mean = get_file(parameter,val[1][0],val[1][1],val[1][2],'Mean',yr_end,yr_st)
    ds_mean = xr.open_dataset(os.path.join(dir_in,filename_mean)) 
    ds_cluster_mean = ds_mean[parameter].where(mask == which_cluster)
    ds_cluster_mean_avg = ds_cluster_mean.NRM_cluster.mean().item(0)
    change_mean = ((ds_cluster_mean_avg-ds_awap_cluster_mean_avg)*100)/ds_awap_cluster_mean_avg   
    #Change in annual max precipitation/runoff
    filename_max = get_file(parameter,val[1][0],val[1][1],val[1][2],'Max',yr_end,yr_st)
    ds_max = xr.open_dataset(os.path.join(dir_in,filename_max)) 
    ds_cluster_max = ds_max[parameter].where(mask == which_cluster)
    ds_cluster_max_avg = ds_cluster_max.NRM_cluster.mean().item(0)
    change_max = ((ds_cluster_max_avg-ds_awap_cluster_max_avg)*100)/ds_awap_cluster_max_avg
    #Change in return period
    filename_annual_max = get_file(parameter,val[1][0],val[1][1],val[1][2],'Annual_Max',yr_end,yr_st)
    ds_annual_max = xr.open_dataset(os.path.join(dir_in,filename_annual_max))
    # Extract annual daily maximum timeseries data for a cluster
    data_gcm = []
    for j in range (0, len(ds_annual_max.time)): 
        annual_max = ts_data(ds_annual_max, parameter, j, which_cluster)
        data_gcm.append(annual_max)
    ts_gcm = pd.DataFrame(data_gcm, columns = ['gcm'])
    gcm_return_period = return_period(ts_gcm, yT)[0]
    change_return_period = ((gcm_return_period-awap_return_period)*100)/awap_return_period
    dd = {'GCM':val[1][0],'Bias Correction':val[1][2],'Emission':val[1][1],'Change in Annual Mean':change_mean,'Change in Annual Max':change_max, 'Change in Return Period':change_return_period}
    cluster_flood_indicator.append(dd)
    break # just want to check one loop/one ensemble